# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [33]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [34]:
# TODO: Import the necessary libs - DONE
import os
from typing import List, Dict, Any, Optional, TypedDict
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions
from tavily import TavilyClient

from lib.agents import Agent, AgentState
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import Tool
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run

In [35]:
# TODO: Load environment variables - DONE
load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Initialize Clients
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"))
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [36]:
# TODO: Create retrieve_game tool - DONE
import re


def retrieve_game(query: str) -> str:
    """Semantic search: Find relevant games in the vector DB."""
    print(f"DEBUG: Retrieving for query: {query}")

    results = collection.query(query_texts=[query], n_results=3)
    docs = results["documents"][0]
    metas = results["metadatas"][0]

    context_parts: List[str] = []
    for doc, meta in zip(docs, metas):
        name = meta.get("Name")
        platform = meta.get("Platform")
        year = meta.get("YearOfRelease")
        context_parts.append(
            f"Game: {name} ({platform}, {year})\n"
            f"Genre: {meta.get('Genre')}\n"
            f"Publisher: {meta.get('Publisher')}\n"
            f"Description: {meta.get('Description')}\n"
        )

    return "\n\n".join(context_parts)


retrieve_tool = Tool(retrieve_game)

#### Evaluate Retrieval Tool

In [37]:
# TODO: Create evaluate_retrieval tool - DONE

import re


def _looks_like_followup(question: str) -> bool:
    q = f" {question.strip().lower()} "
    followup_markers = [
        " it ",
        " its ",
        " this ",
        " that ",
        " this game ",
        " that game ",
        " the game ",
    ]
    return any(m in q for m in followup_markers)


def _is_time_sensitive(question: str) -> bool:
    q = question.lower()
    return any(k in q for k in ["latest", "news", "current", "today", "recent", "update", "updates"])


def _normalize(text: str) -> str:
    return re.sub(r"\s+", " ", (text or "").lower()).strip()


def _keyword_overlap(question: str, retrieved_docs: str) -> bool:
    q = _normalize(question)
    d = _normalize(retrieved_docs)

    stop = {
        "what", "when", "where", "who", "which", "is", "was", "were", "a", "an", "the",
        "for", "to", "of", "on", "in", "and", "or", "about", "released", "release",
        "game", "tell", "me", "it", "its", "this", "that",
    }

    words = [w for w in re.findall(r"[a-z0-9']+", q) if w not in stop and len(w) >= 3]

    # If question contains an obvious franchise token, require it in retrieved docs.
    franchise_tokens = ["mario", "pokemon", "gran", "turismo", "gta", "ragnarok", "mortal", "kombat"]
    if any(t in q for t in franchise_tokens):
        return any(t in d for t in franchise_tokens if t in q)

    # Otherwise require some overlap on meaningful words.
    return any(w in d for w in words)


def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """Decide if the retrieved docs are sufficient to answer the question."""
    print("DEBUG: Evaluating context")

    if not retrieved_docs or "Game:" not in retrieved_docs:
        return "NO"

    # Time-sensitive requests should go to web.
    if _is_time_sensitive(question):
        return "NO"

    # Special handling: GTA 6 should not be answered from older GTA entries.
    qn = _normalize(question)
    if "gta 6" in qn or "gta vi" in qn or "grand theft auto 6" in qn or "grand theft auto vi" in qn:
        dn = _normalize(retrieved_docs)
        if ("6" not in dn) and ("vi" not in dn):
            return "NO"

    # Follow-ups ("it/this game") are acceptable if we retrieved any specific game block.
    if _looks_like_followup(question):
        return "YES"

    # Heuristic overlap check
    if _keyword_overlap(question, retrieved_docs):
        return "YES"

    # LLM fallback (robust prompt)
    llm = LLM(model="gpt-3.5-turbo", temperature=0.0)
    prompt = f"""
You are an evaluator for a RAG agent.

Decide if the Retrieved Docs contain enough information to answer the Question.

Guidance:
- Answer YES if the docs mention the same game/franchise and include the needed fact.
- Answer NO if the docs are about a different game/franchise OR if the question asks for time-sensitive "latest news".
- Return ONLY: YES or NO

Question: {question}
Retrieved Docs:
{retrieved_docs}
"""

    response = llm.invoke([UserMessage(content=prompt)])
    return (response.content or "").strip().upper()


evaluate_tool = Tool(evaluate_retrieval)

#### Game Web Search Tool

In [38]:
# TODO: Create game_web_search tool - DONE
def game_web_search(question: str) -> str:
    """
    Semantic search: Finds most results in the vector DB (Fallback to web)
    """
    print(f"DEBUG: Web searching for: {question}")
    try:
        response = tavily_client.search(question, search_depth="advanced")
        context = ""
        for result in response['results']:
            context += f"Title: {result['title']}\nContent: {result['content']}\nURL: {result['url']}\n\n"
        return context
    except Exception as e:
        return f"Error searching web: {e}"

search_tool = Tool(game_web_search)

### Agent

In [39]:
# TODO: Create your Agent abstraction using StateMachine - DONE
import re

class ResearchAgentState(AgentState):
    retrieved_context: Optional[str]
    evaluation_result: Optional[str]
    source: Optional[str]

class ResearchAgent(Agent):
    def __init__(self, model_name="gpt-3.5-turbo", temperature=0.7):
        super().__init__(model_name, "You are a game research assistant.", temperature=temperature)
    
    def _extract_game_from_history(self, messages) -> Optional[str]:
        """Extract the most recently mentioned game from conversation history."""
        for msg in reversed(messages):
            if isinstance(msg, AIMessage):
                content = msg.content
                # Look for known game patterns
                patterns = [
                    r'(Super Mario 64)',
                    r'(Gran Turismo(?:\s*\d*)?)',
                    r'(Pokémon [A-Za-z\s]+)',
                    r"(Marvel's Spider-Man(?:\s*\d*)?)",
                    r'(Grand Theft Auto[:\s]+[A-Za-z\s]+)',
                    r'(Minecraft)',
                    r'(Wii Sports)',
                    r'(Mario Kart[:\s]*\d*[A-Za-z\s]*)',
                ]
                for pattern in patterns:
                    match = re.search(pattern, content, re.IGNORECASE)
                    if match:
                        return match.group(1).strip()
        return None
    
    def _is_followup_query(self, query: str) -> bool:
        """Check if query is a follow-up referencing previous context."""
        followup_indicators = [
            'it', 'this game', 'that game', 'the game', 'its', 'this',
            'who is the main', 'what year', 'when was it', 'was it', 'what platform'
        ]
        query_lower = query.lower()
        for indicator in followup_indicators:
            if indicator in query_lower:
                return True
        return False
    
    def _format_conversation_history(self, messages) -> str:
        """Format conversation history for context."""
        history_parts = []
        for msg in messages:
            if isinstance(msg, AIMessage):
                history_parts.append(f"Previous answer: {msg.content[:300]}")
        return "\n".join(history_parts[-3:]) if history_parts else ""
        
    def _retrieve_step(self, state: ResearchAgentState) -> ResearchAgentState:
        query = state["user_query"]
        messages = state.get("messages", [])
        
        # Check if this is a follow-up query and expand it with game name from context
        if self._is_followup_query(query):
            game_name = self._extract_game_from_history(messages)
            if game_name:
                query = f"{game_name}: {query}"
                print(f"DEBUG: Expanded follow-up to: {query}")
        
        print(f"DEBUG: Retrieving for query: {query}")
        context = retrieve_game(query)
        return {**state, "retrieved_context": context, "source": "internal"}

    def _evaluate_step(self, state: ResearchAgentState) -> ResearchAgentState:
        query = state["user_query"]
        context = state["retrieved_context"]
        print(f"DEBUG: Evaluating context")
        result = evaluate_retrieval(query, context)
        return {**state, "evaluation_result": result}

    def _web_search_step(self, state: ResearchAgentState) -> ResearchAgentState:
        query = state["user_query"]
        print(f"DEBUG: Web searching for: {query}")
        context = game_web_search(query)
        return {**state, "retrieved_context": context, "source": "web"}

    def _generate_answer_step(self, state: ResearchAgentState) -> ResearchAgentState:
        query = state["user_query"]
        context = state["retrieved_context"]
        source = state["source"]
        messages = state.get("messages", [])
        
        # Include conversation history for LLM context
        history = self._format_conversation_history(messages)
        history_section = f"\nConversation History:\n{history}\n" if history else ""
        
        prompt_content = f"""You are a game research assistant. Answer based on the context.
{history_section}
Context ({source}): 
{context}

Question: {query}

Instructions:
- Provide a complete answer in 2-4 sentences
- If a follow-up, use conversation history to determine which game
- Include: Game Name (Platform, Year)
"""
        
        new_user_message = UserMessage(content=prompt_content)
        messages_for_llm = messages + [new_user_message]
        
        llm = LLM(model=self.model_name, temperature=self.temperature)
        response = llm.invoke(messages_for_llm)
        
        return {**state, "messages": messages + [new_user_message, AIMessage(content=response.content)]}

    def _create_state_machine(self) -> StateMachine[ResearchAgentState]:
        machine = StateMachine[ResearchAgentState](ResearchAgentState)
        
        entry = EntryPoint[ResearchAgentState]()
        retrieve = Step[ResearchAgentState]("retrieve", self._retrieve_step)
        evaluate = Step[ResearchAgentState]("evaluate", self._evaluate_step)
        web_search = Step[ResearchAgentState]("web_search", self._web_search_step)
        generate = Step[ResearchAgentState]("generate", self._generate_answer_step)
        termination = Termination[ResearchAgentState]()
        
        machine.add_steps([entry, retrieve, evaluate, web_search, generate, termination])
        
        machine.connect(entry, retrieve)
        machine.connect(retrieve, evaluate)
        
        def check_eval(state: ResearchAgentState) -> str:
            return "generate" if "YES" in state["evaluation_result"] else "web_search"
        
        machine.connect(evaluate, [generate, web_search], condition=check_eval)
        machine.connect(web_search, generate)
        machine.connect(generate, termination)
        
        return machine

In [40]:
# TODO: Invoke your agent - DONE
agent = ResearchAgent()

queries = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X realeased for Playstation 5?"
]

for query in queries:
    print(f"\n--- Query: {query} ---")
    run = agent.invoke(query)
    print(f"Answer: {run.get_final_state()['messages'][-1].content}")


--- Query: When Pokémon Gold and Silver was released? ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: When Pokémon Gold and Silver was released?
DEBUG: Retrieving for query: When Pokémon Gold and Silver was released?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Answer: Pokémon Gold and Silver was released for the Game Boy Color in 1999.

--- Query: Which one was the first 3D platformer Mario game? ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: Which one was the first 3D platformer Mario game?
DEBUG: Retrieving for query: Which one was the first 3D platformer Mario game?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
[StateMachine] Executing step: generate
[StateMachine] Terminating: __terminat

### (Optional) Advanced

In [15]:
# TODO: Update your agent with long-term memory - DONE
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes - DONE

class LearningResearchAgent(ResearchAgent):
    def _generate_answer_step(self, state: ResearchAgentState) -> ResearchAgentState:
        # Generate answer using parent logic
        new_state = super()._generate_answer_step(state)
        
        # Long-term memory: Save web results to ChromaDB
        if new_state["source"] == "web":
            print(f"DEBUG: Memorizing new info about '{new_state['user_query']}'")
            try:
                collection.add(
                    ids=[f"memory_{abs(hash(new_state['user_query']))}"],
                    documents=[f"Q: {new_state['user_query']}\nA: {new_state['messages'][-1].content}"],
                    metadatas=[{"Name": "Learned Memory", "Platform": "Web", "YearOfRelease": 2025}]
                )
            except Exception as e:
                print(f"DEBUG: Memory update skipped: {e}")
        return new_state

# Test the learning agent
learning_agent = LearningResearchAgent()
learning_agent.invoke("What is the release date of GTA 6?")

[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: What is the release date of GTA 6?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: What is the release date of GTA 6?
[StateMachine] Executing step: web_search
DEBUG: Memorizing new info about 'What is the release date of GTA 6?'
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__


Run('335fbdc5-d717-4698-9cef-abde152d5cff')

In [13]:
# Initialize Agent
agent = ResearchAgent()

# Test Queries
queries = [
    "Who developed Gran Turismo?",
    "When was God of War Ragnarok released?",
    "What is the latest news about GTA 6?"
]

for query in queries:
    print(f"\n--- Query: {query} ---")
    run = agent.invoke(query)
    final_state = run.get_final_state()
    print(f"Final Answer:\n{final_state['messages'][-1].content}")
    print(f"Source: {final_state.get('source')}")


--- Query: Who developed Gran Turismo? ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: Who developed Gran Turismo?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Final Answer:
Gran Turismo was developed by Sony Computer Entertainment. (Source: Context)
Source: internal

--- Query: When was God of War Ragnarok released? ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: When was God of War Ragnarok released?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
DEBUG: Web searching for: When was God of War Ragnarok released?
[StateMachine] Executing step: web_search
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Final Answer:
God of War Ragnarok was released on November 9, 2022. (Source: https://www.playstation.com/en-us/

In [41]:
# Demonstrate Session Management with games in the database
print("\n=== Session Management Demo ===")

# Session 1: Ask about Super Mario 64 (in database - 009.json)
print("\n--- Session 1: Super Mario 64 Context ---")
agent.invoke("Tell me about Super Mario 64.", session_id="session_1")
run1 = agent.invoke("What year was it released?", session_id="session_1")
print(f"Session 1 Follow-up: {run1.get_final_state()['messages'][-1].content}")

# Session 2: Ask about Gran Turismo (in database - 001.json)  
print("\n--- Session 2: Gran Turismo Context ---")
agent.invoke("Tell me about Gran Turismo.", session_id="session_2")
run2 = agent.invoke("What year was it released?", session_id="session_2")
print(f"Session 2 Follow-up: {run2.get_final_state()['messages'][-1].content}")

# Verify Session 1 context is preserved (should still reference Super Mario 64)
print("\n--- Session 1: Context Verification ---")
run3 = agent.invoke("Who is the main character in this game?", session_id="session_1")
print(f"Session 1 Verification: {run3.get_final_state()['messages'][-1].content}")

# Verify Session 2 stays on Gran Turismo (no cross-contamination from Session 1)
print("\n--- Session 2: Independent Context Verification ---")
run4 = agent.invoke("What platform is this game on?", session_id="session_2")
print(f"Session 2 Verification: {run4.get_final_state()['messages'][-1].content}")


=== Session Management Demo ===

--- Session 1: Super Mario 64 Context ---
[StateMachine] Starting: __entry__
DEBUG: Retrieving for query: Tell me about Super Mario 64.
DEBUG: Retrieving for query: Tell me about Super Mario 64.
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
DEBUG: Expanded follow-up to: Super Mario 64: What year was it released?
DEBUG: Retrieving for query: Super Mario 64: What year was it released?
DEBUG: Retrieving for query: Super Mario 64: What year was it released?
[StateMachine] Executing step: retrieve
DEBUG: Evaluating context
DEBUG: Evaluating context
[StateMachine] Executing step: evaluate
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
Session 1 Follow-up: Super Mario 64 was released in 1996 for the Nintendo 64.

--- Sess